In [1]:
# conda env: st(Python 3.12.2)
import os
import sys
from datacat4ml.const import DATA_DIR, FETCH_DATA_DIR, FETCH_FIG_DIR, FEATURIZE_DATA_DIR, FEATURIZE_FIG_DIR

import os
import pandas as pd

#  Load the data

In [18]:
# load the dataset
or_chembl_id = ['CHEMBL233', 'CHEMBL237', 'CHEMBL236', 'CHEMBL2014']
gpcr_ki = pd.read_csv(os.path.join(DATA_DIR, 'data_prep', '1_data_fetch', 'ki_maxcur_8_data.csv'))
print(f"The shape of gpcr_ki is: {gpcr_ki.shape}")
gpcr_ic50 = pd.read_csv(os.path.join(DATA_DIR, 'data_prep', '1_data_fetch', 'ic50_maxcur_8_data.csv'))
print(f"The shape of gpcr_ic50 is: {gpcr_ic50.shape}")
gpcr_ec50 = pd.read_csv(os.path.join(DATA_DIR, 'data_prep', '1_data_fetch', 'ec50_maxcur_8_data.csv'))
print(f"The shape of gpcr_ec50 is: {gpcr_ec50.shape}")
# extract the rows where the 'target_chembl_id' is one of the elements in the list OR_chembl_id
or_ki = gpcr_ki[gpcr_ki['target_chembl_id'].isin(or_chembl_id)]
print(f"The shape of or_ki is: {or_ki.shape}")

/tmp/ipykernel_3631220/1533418957.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  gpcr_ki = pd.read_csv(os.path.join(DATA_DIR, 'data_prep', '1_data_fetch', 'ki_maxcur_8_data.csv'))


The shape of gpcr_ki is: (139416, 28)


/tmp/ipykernel_3631220/1533418957.py:5: DtypeWarning: Columns (7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  gpcr_ic50 = pd.read_csv(os.path.join(DATA_DIR, 'data_prep', '1_data_fetch', 'ic50_maxcur_8_data.csv'))


The shape of gpcr_ic50 is: (88622, 28)
The shape of gpcr_ec50 is: (56294, 28)
The shape of or_ki is: (13533, 28)


/tmp/ipykernel_3631220/1533418957.py:7: DtypeWarning: Columns (7,13) have mixed types. Specify dtype option on import or set low_memory=False.
  gpcr_ec50 = pd.read_csv(os.path.join(DATA_DIR, 'data_prep', '1_data_fetch', 'ec50_maxcur_8_data.csv'))


In [3]:
gpcr_ki_assay_desc = gpcr_ki['assay_desc'].values.tolist()

In [65]:
gpcr_ki.columns

Index(['assay_id', 'assay_chembl_id', 'tid', 'target_chembl_id',
       'standard_type', 'pchembl_value', 'assay_type', 'assay_category',
       'assay_organism', 'assay_tax_id', 'assay_strain', 'assay_tissue',
       'assay_cell_type', 'assay_subcellular_fraction', 'bao_format',
       'variant_id', 'assay_test_type', 'assay_desc', 'cell_id', 'tissue_id',
       'curated_by', 'relationship_type', 'aidx', 'confidence_score',
       'molregno', 'compound_chembl_id', 'canonical_smiles',
       'assay_info_hash'],
      dtype='object')

# Implement the transformer

using API in transformers to build simpletransfomers-like architecture


## Hugging Face > DEVELOPER GUIDES > Notebooks with examples

- https://huggingface.co/docs/transformers/en/notebooks#community-notebooks

### PyTorch Examples > Natural Language Processing

- Train your tokenizer (done!)
- Train your language model (done!)
- How to fine-tune a model on text classification (done!)
- How to fine-tune a model on language modeling
- how to train a language model from scratc (done!)
- Reformer 

#### how to train a language model from scratch

- https://github.com/huggingface/blog/blob/main/notebooks/01_how_to_train.ipynb
-  As we are training from scratch, we only initialize `model` from a config, not from an existing pretrained model or checkpoint

##### Train a tokenizer

In [11]:
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer

# initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# customize training
tokenizer.train(files=['gpcr_assay_descs.txt'], vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

In [13]:
tokenizer.save_model("from_scratch")

['from_scratch/vocab.json', 'from_scratch/merges.txt']

In [14]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

tokenizer = ByteLevelBPETokenizer(
    "./from_scratch/vocab.json",
    "./from_scratch/merges.txt",
)

tokenizer.enable_truncation(max_length=512)
tokenizer.encode(gpcr_ki_assay_desc[0])

Encoding(num_tokens=17, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [15]:
tokenizer.encode(gpcr_ki_assay_desc[0]).tokens

['Displacement',
 'Ġof',
 'Ġ[',
 '3',
 'H',
 ']',
 'EK',
 'Ġfrom',
 'ĠOpioid',
 'Ġreceptor',
 'Ġdelta',
 'Ġ1',
 'Ġin',
 'Ġguinea',
 'Ġpig',
 'Ġbrain',
 'Ġmembrane']

--> how to set the word should not be split

##### Train a language model from scratch

In [8]:
import torch
torch.cuda.is_available()

False

In [16]:
# define the following config for the model
from transformers import RobertaConfig
config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

# recreate the tokenizer in transformers
# It is necessary because it ennsures compatibility between the tokenizer and the transformer model. 
# The `transformers` provides additional features and integration that may not be available in the `tokenizers` library alone
from transformers import RobertaTokenizerFast
tokenizer = RobertaTokenizerFast.from_pretrained("./from_scratch", max_len=512)

# initialize the model.
# As we are training from scratch, we only initialize from a config, not from an existing pretrained model or checkpoint.

from transformers import RobertaForMaskedLM
model = RobertaForMaskedLM(config=config)
model.num_parameters()


/storage/homefs/yc24j783/miniconda3/envs/st/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


83504416

In [17]:
# build training dataset
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./gpcr_assay_descs.txt",
    block_size=128,
)

# batch the data
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

# initialize the trainer
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="./from_scratch",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# start training
trainer.train()

/storage/homefs/yc24j783/miniconda3/envs/st/lib/python3.12/site-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

### Other modalities

- Probabilistic Time Series Forecasting

### Utility notebooks

- How to use Benchmarks

## Github repo: huggingface/transformers

- ref
https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_classification.py

In [5]:
import logging
import os
import random
import sys
from dataclasses import dataclass, field
from typing import List, Optional

import datasets
#import evaluate
import numpy as np
from datasets import Value, load_dataset

import transformers
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version

## Github repo: huggingface/notebooks/examples (done:)

### text_classification.ipynb

- https://github.com/huggingface/notebooks/blob/main/examples/text_classification.ipynb

In [19]:
# this notebook is built on to run on any of the tasks in the list below, 
GPCR_TASKS = ["GPCR_KI", "GPCR_IC50", "GPCR_EC50"]
# with any model checkpoint from the Model Hub as long as that model has a version with a classification head. 
# Depending on your model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors.
task = "GPCR_KI"
model_checkpoint = "from_scratch"
batch_size = 16

#### Loading the dataset

or

##### set the threshold between active and inactive

adopt two thresholding mechanisms developed in FS-Mol: 

1. fixted: applies pKX = 5(pchembl_value)/ 50% as a threshold
2. autothreshold:  finds the median of the data and accepts it as a threshold provided it is in the range pKI 4-6 or 5-7 for enzymes. 

For specific tasks in a different phase of drug discovery we recommend implementing a hand-tuned threshold.(as long as the the active ratio falls between 0.30 and 0.70)

In [ ]:
import math
standard_unit_set = {"nM", "%", "uM"}
from typing import Optional, Tuple, Callable

class MolError(Exception):
    """Custom exception for failure to convert molecule."""

    def __init__(self, smiles: str, error: str):
        super().__init__()
        self._smiles = smiles
        self._error = error

    def __str__(self):
        return f"Cannot make rdkit mol from SMILES: {self._smiles} \n {self._error}"


def clean_units(x: pd.Series) -> bool:
    """Remove measurements that have units outside the permitted set"""
    return x["standard_units"] not in standard_unit_set


def clean_values(x: pd.Series) -> bool:
    return np.isnan(x["standard_value"])


def log_standard_values(x: pd.Series) -> float:
    """Convert standard value to -log10([C]/nM)"""
    if x["standard_value"] < 1e-13 or np.isnan(x["standard_value"]):
        return float("NaN")
    else:
        return -1 * math.log10(x["standard_value"] * 10 ** -9)

relation_set_lessthan = {"<", "<="}
relation_set_morethan = {">", ">="}
relation_equals = {"=", "~"}

def activity_threshold(x: float, threshold: float, buffer: float = 0.5) -> str:

    """
    Apply a threshold to activity measurements.

    For example, IC50/ EC50 measurements.
    """

    # for now just use the passed median value to threshold
    # this is calculated from the entire dataframe.
    val = x["log_standard_value"]
    relation = x["standard_relation"]

    # note: standard relations apply to standard value not log standard
    if val >= (threshold + buffer):
        return "active"
    elif val > threshold and val < (threshold + buffer) and relation in relation_set_lessthan:
        return "active"
    elif (
        val > threshold
        and val < (threshold + buffer)
        and relation in relation_set_morethan.union(relation_equals)
    ):
        return "weak active"
    elif (
        val > (threshold - buffer)
        and val <= threshold
        and relation in relation_set_lessthan.union(relation_equals)
    ):
        return "weak inactive"
    elif val > (threshold - buffer) and val <= threshold and relation in relation_set_morethan:
        return "inactive"
    elif val <= (threshold - buffer):
        return "inactive"

def autothreshold(x: pd.Series) -> Tuple[pd.DataFrame, float]:

    """
    Apply autothesholding procedure to data:

    1) Find the median for an assay
    2) Use the median as a threshold if it sits within the required range.
    For enzymes this is 5 <= median(pXC) <= 7, for everything else this is
    4 <= median(pXC) <= 6. Inhibition measurements use <= 50% as medians above
    this are found with assays containing almost entirely strong inhibitors.
    If the median is outside the required range, fix to pXC = 5.0 or inhibition = 50%
    3) Apply the threshold to the data series.

    For activity measurements, log standard value is used.
    """

    df = pd.DataFrame(x)

    
    threshold_limits = (5, 7)
    # get median
    median = df["log_standard_value"].median()
    threshold = median
    buffer = df["log_standard_value"].std() / 10

    # use as a threshold provided it is in a sensible
    # range. This was chosen as pKI 4-6 in general, 5-7 for enzymes
    if "protein_class_desc" in df.columns:
        if any(("enzyme" in x) or ("ase" in x) for x in df.protein_class_desc.values):
            threshold_limits = (5, 7)
        else:
            threshold_limits = (4, 6)
    else:
        threshold_limits = (4, 6)

    if median < threshold_limits[0] or median > threshold_limits[1]:
        threshold = 5.0
    else:
        threshold = median

    df["activity_string"] = df.apply(
        activity_threshold, args=(threshold,), buffer=buffer, axis=1
    )

    return df, threshold

In [36]:
from datasets import Dataset, load_metric

# extract the column 'assay_desc' and 'pchembl_value' from the dataset and generate a new dataframe
or_df = or_ki[['assay_desc', 'pchembl_value']].reset_index(drop=True)
dataset = Dataset.from_pandas(or_df)

In [37]:
dataset.train_test_split(test_size=0.2)
# assign the training and testing dataset to the variables train_dataset and test_dataset
train_ds = dataset.train_test_split(test_size=0.2)['train']
test_ds = dataset.train_test_split(test_size=0.2)['test']

In [44]:
for column, typ in dataset.features.items():
    print(f'column: {column}, type: {typ}')

column: assay_desc, type: Value(dtype='string', id=None)
column: pchembl_value, type: Value(dtype='float64', id=None)


In [38]:
print(f'The train_ds is: \n{train_ds}\n')
print(f'The test_ds is: \n{test_ds}')

The train_ds is: 
Dataset({
    features: ['assay_desc', 'pchembl_value'],
    num_rows: 10826
})

The test_ds is: 
Dataset({
    features: ['assay_desc', 'pchembl_value'],
    num_rows: 2707
})


In [46]:
# To get a sense of what the data look like, the following function will show some examples picked randomly from the dataset

import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
    
show_random_elements(train_ds)

,assay_desc,pchembl_value
0,Displacement of [3H]Naltindole from human delta opioid receptor expressed in CHO cell membrane,6.72
1,Activity was evaluated by inhibition of the binding of 1 nM [3H]DPDPE at Opioid receptor delta 1 binding site,5.74
2,Displacement of [3H]DPN from kappa opioid receptor expressed in CHO cells after 1 hr,8.80
3,Binding affinity against human Opioid receptor kappa 1 on CHO cell membranes by [3H]U-69593 displacement.,6.76
4,Binding affinity to mu opioid receptor (unknown origin),8.52
5,Displacement of [3H]-DPDPE from human DOR expressed in CHO cell membranes measured after 2 hrs by microbeta scintillation counting method,6.16
6,Displacement of [3H]DAMGO from mu opioid receptor expressed in CHO cells,10.53
7,Displacement of [3H]C1-DPDPE from human recombinant Opioid receptor delta 1 on CHO cell membranes.,7.99
8,"Displacement of [3H]U69,593 from human kappa opioid receptor expressed in CHO cells after 2 hrs by liquid scintillation counting",6.52
9,Displacement of [3H]DPDPE from human delta opioid receptor expressed in HN9.10 cells,7.44


In [54]:
# metric = load_metric("accuracy")
# --> is it okay to directly import metrics implemented in sklearn?

In [57]:
train_ds[0]

{'assay_desc': 'Displacement of [3H]DAMGO from mu opioid receptor (unknown origin) expressed in HEK293 cells after 1 hr by liquid scintillation counting',
 'pchembl_value': 10.96}

#### Preprocessing the data

In [58]:
# This is to tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put them in a format the model expects.

# here we use the tokenizer we trained by gpcr_assay_descs.txt before.
tokenizer = RobertaTokenizerFast.from_pretrained("./from_scratch", max_len=512)

# directly call this tokenizer on one sentecne or a pair of sentences
tokenizer("Displacement of [3H]Naltindole from human delta opioid receptor expressed in CHO cell membrane", "Activity was evaluated by inhibition of the binding of 1 nM [3H]DPDPE at Opioid receptor delta 1 binding site	")

/storage/homefs/yc24j783/miniconda3/envs/st/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


{'input_ids': [0, 310, 285, 292, 23, 44, 65, 6332, 305, 289, 727, 530, 287, 311, 275, 331, 297, 478, 2, 2, 3053, 416, 1063, 321, 590, 285, 386, 369, 285, 328, 511, 292, 23, 44, 65, 1217, 402, 970, 287, 727, 328, 369, 2364, 202, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [62]:
def preprocess_funcition(examples):
    return tokenizer(examples['assay_desc'], truncation=True)

# To apply this function on all the sentences in the dataset, we use the map method of `datasest` object we created earlier. 
# This will apply the function on all the elements of all the splits in the `dataset`, 
# so our training and validation splits will be preprocessed in one single command.
encoded_train_ds = train_ds.map(preprocess_funcition, batched=True)
encoded_test_ds = test_ds.map(preprocess_funcition, batched=True)

Map: 100%|██████████| 2707/2707 [00:00<00:00, 24129.43 examples/s]


In [63]:
encoded_train_ds[0]

{'assay_desc': 'Displacement of [3H]DAMGO from mu opioid receptor (unknown origin) expressed in HEK293 cells after 1 hr by liquid scintillation counting',
 'pchembl_value': 10.96,
 'input_ids': [0,
  310,
  285,
  292,
  23,
  44,
  65,
  862,
  305,
  716,
  530,
  287,
  342,
  579,
  578,
  13,
  311,
  275,
  363,
  380,
  317,
  381,
  328,
  428,
  321,
  465,
  372,
  382,
  2],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

#### Fine-tuning the model

In [64]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
num_labels = 2 # binary classification
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels) # failed because model_checkpoint is not a pretrained model

OSError: from_scratch does not appear to have a file named config.json. Checkout 'https://huggingface.co/from_scratch/tree/None' for available files.

In [66]:
# The warning is telling us we are throwing away some weights (the vocab_transform and vocab_layer_norm layers) 
# and randomly initializing some other (the pre_classifier and classifier layers). 
# This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective 
# and replacing it with a new head for which we don't have pretrained weights, 
# so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

In [ ]:
# To initialize a 'Trainer', the most important is the `TrainingArguments`, which is a class that contains all the attributes to customize the training.
# It requires one folder name, here is `f"{model_name}-finetuned-{task}"``
args = TrainingArguments(
    f"{model_name}-finetuned-{task}", 
    evaluation_strategy = "epoch", #
    save_strategy = "epoch", #
    learning_rate=2e-5, #
    per_device_train_batch_size=batch_size, #
    per_device_eval_batch_size=batch_size, #
    num_train_epochs=5, #
    weight_decay=0.01, #
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,#
    push_to_hub=True,
)

Here we set 
- the evaluation to be done at the end of each epoch, 
- tweak the learning rate, 
- use the batch_size defined at the top of the notebook 
- and customize the number of epochs for training, 
- as well as the weight decay. 
- Since the best model might not be the one at the end of training, we ask the Trainer to load the best model it saved (according to metric_name) at the end of training.
- The last argument to setup everything so we can push the model to the Hub regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. 

If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the hub_model_id argument to set the repo name (it needs to be the full name, including your namespace: for instance "sgugger/bert-finetuned-mrpc" or "huggingface/bert-finetuned-mrpc").

The last thing to define for our Trainer is how to compute the metrics from the predictions. We need to define a function for this, which will just use the metric we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits (our just squeeze the last axis in the case of STS-B):

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# pass all of this along with the datasets to the `Trainer`
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_train_ds,
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# You might wonder why we pass along the tokenizer when we already preprocessed our data. 
# This is because we will use it once last time to make all the samples we gather the same length by applying padding, 
# which requires knowing the model's preferences regarding padding (to the left or right? with which token?). 
# The tokenizer has a pad method that will do all of this right for us, and the Trainer will use it. 
# You can customize this part by defining and passing your own data_collator which will receive the samples like the dictionaries seen above and will need to return a dictionary of tensors.

# now finetune thid model by just calling the `train` method
trainer.train()
# we can check with the `evaluate` method that our `Trainer` did reload the best model properly (if it was not the last one)
trainer.evaluate()

#### Hyperparameter search

The `trainer` supports hyperparameter search using `optuna`

`! pip install optuna`

In [ ]:
# During hyperparameter search, the Trainer will run several trainings, 
# so it needs to have the model defined via a function (so it can be reinitialized at each new run) 
# instead of just having it passed. We jsut use the same function as before:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

# and we can instantiate our `Trainer` like before
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=encoded_train_ds,
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)`

best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize") 
# n_trails is the number of trial runs to rest, 
# direction for single objective optimization, can be 'minimize' or 'maximize', you can pick 'minimize' when optimizing the validation loss.

# To reproduce the best training,
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)
trainer.train()

### language modeling_from_scratch.ipynb

- https://github.com/huggingface/notebooks/blob/main/examples/language_modeling_from_scratch.ipynb

### tokenizer_training.ipynb

- https://github.com/huggingface/notebooks/blob/main/examples/tokenizer_training.ipynb

#### Using an existing tokenizer

train a tokenizer with the exact same algorithms and parameters as an exxisting one using `train_new_from_iterator` API

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")


tokenizer.is_fast

new_tokenizer = tokenizer.train_new_from_iterator(batch_iterator(), vocab_size=25000)

#### Building your tokenizer from scratch

train a tokenizer that doesn't look like anything in existence, you will need to build it block by block, including the below steps:
- normalization
- pre-tokenization
- model
- post-processing
- decoding

##### WordPiece model like BERT

##### BPE model like GPT-2

this is what david used in Enzymatic prediction project

In [ ]:
tokenizer = Tokenizer(models.BPE())

tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

tokenizer.pre_tokenizer.pre_tokenize_str("This is an example!")

trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>"])
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
tokenizer.decoder = decoders.ByteLevel()


from transformers import GPT2TokenizerFast

new_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer)